This notebook uses cvxpy library (it has been installed on urbana package, so it should be updated in order to use the entire program)

In [1]:
%load_ext lab_black

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys

sys.path.append(
    "/Users/mateo/Documents/Máster Big Data/TFM/codes/urbana-barcelona-master/src"
)

In [4]:
import os
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import seaborn as sns

from urbana.models.plot_predictions import PredictedAccuracy
from urbana.constants import DIR_REPO, DIR_DATA

In [5]:
# papermill parameters cell
# https://papermill.readthedocs.io/en/latest/usage-parameterize.html

YEAR = 2023  # ALLOWED_YEARS = [2017, 2018, 2023]
MONTH = 12

OUTPUT_WARNINGS = False
SAVE_FIGS = False
FIT_INTERCEPT = True

VARIABLE_TO_PREDICT = "Airbnb_Number"

# Number of selected variables in these categories
K_EDUCATION = 1
K_AGE = 2
K_NATIONALITY = 2
K_RENT = 1
K_POI = 10

# Warnings
if not OUTPUT_WARNINGS:
    import warnings

    warnings.filterwarnings("ignore")

# Parameter check
ALLOWED_YEARS = [2017, 2018, 2023]

if YEAR not in ALLOWED_YEARS:
    raise Exception("Please select a year within: {}".format(ALLOWED_YEARS))

if YEAR == 2018 and MONTH == 3:
    raise Exception(
        "Month 3 (March) is not available for 2018. Please choose a different month."
    )

# Create folders to store the data
DIR_VAR = DIR_DATA / "processed/{}".format(VARIABLE_TO_PREDICT)
DIR_MONTH = DIR_VAR / "{}_{:02d}".format(YEAR, MONTH)
DIR_GWSVR = DIR_MONTH / "06_gwsvr"

if SAVE_FIGS:
    folder_list = [
        DIR_GWSVR,
        DIR_GWSVR / "coefficients",
    ]
    for folder in folder_list:
        if not os.path.exists(folder):
            os.makedirs(folder)

PATH_TO_FILE = DIR_DATA / "interim/sections_{}_{:02d}.csv".format(YEAR, MONTH)
if os.path.isfile(PATH_TO_FILE) is False:
    raise Exception(
        'Please run first the notebook "00acquisition.ipynb" with the same date and "SAVE_DATA" set to True'
    )

PATH_TO_FILE = DIR_MONTH / "01_linear/coefficients.csv"
if os.path.isfile(PATH_TO_FILE) is False:
    raise Exception(
        'Please run first the notebook "01linear.ipynb" with the same date and "SAVE_MODEL" set to True'
    )

# Data Import

In [6]:
sect = pd.read_csv(DIR_DATA / "interim/sections_{}_{:02d}.csv".format(YEAR, MONTH))
sect.set_index("Tag", inplace=True)
sect.drop(
    [
        "N_district",
        "N_neighbourhood",
        "N_section",
        "Airbnb_Price",
        "Airbnb_Price_Person",
        "Airbnb_Location_Score",
        "Percentage_Age_25_39",
    ],
    axis=1,
    inplace=True,
)

X = sect.drop(VARIABLE_TO_PREDICT, axis=1)
y = np.array(sect[VARIABLE_TO_PREDICT]).reshape((-1, 1))

geo_info = gpd.read_file(DIR_DATA / "interim/sections_geo.json")
geo_info.set_index("Tag", inplace=True)

geo_coefs = geo_info.copy()
geo_tvals = geo_info.copy()
geo_nans = geo_info.copy()

geo_info[VARIABLE_TO_PREDICT] = sect[VARIABLE_TO_PREDICT]

coords = np.column_stack(
    [geo_info["geometry"].centroid.x, geo_info["geometry"].centroid.y]
)

geo_info["centroid"] = geo_info["geometry"].centroid

# 1st model: All features

GWSVR 

For each section $j$: 

$$
\min_{\mathbf{w}, b, \xi, \xi^*} \frac{1}{2} \|\mathbf{w}\|^2 + \sum_{i=1}^n (\xi_i + \xi_i^*) \rho_{ij}
$$

subject to:

$$
\begin{align*}
y_i - \mathbf{w}^T \mathbf{x}_i - b &\leq \epsilon + \xi_i \\
\mathbf{w}^T \mathbf{x}_i + b - y_i &\leq \epsilon + \xi_i^* \\
\xi_i, \xi_i^* &\geq 0
\end{align*}
$$


In [7]:
# import cvxpy as cp
# from scipy.spatial.distance import cdist
# from sklearn.metrics import mean_squared_error
# from joblib import Parallel, delayed

# centroids = geo_info.centroid
# centroids = np.array([[point.x, point.y] for point in geo_info["centroid"]])

# distances = cdist(centroids, centroids, "euclidean")


# # Function to compute Gaussian weights
# def compute_gaussian_weights(distances, bandwidth, j):
#     weights = np.exp(-0.5 * (distances[j, :] / bandwidth) ** 2)
#     return weights.reshape(-1, 1)


# def fit_svr_with_weights(X_train, y_train, weights, C=1024, epsilon=0.1):
#     w = cp.Variable((X_train.shape[1], 1))
#     b = cp.Variable()
#     xi_pos = cp.Variable((X_train.shape[0], 1))
#     xi_neg = cp.Variable((X_train.shape[0], 1))

#     objective = cp.Minimize(
#         0.5 * cp.sum_squares(w) + cp.sum(cp.multiply(weights, xi_pos + xi_neg))
#     )

#     constraints = [
#         y_train.reshape(-1, 1) - cp.matmul(X_train, w) - b <= epsilon + xi_pos,
#         cp.matmul(X_train, w) + b - y_train.reshape(-1, 1) <= epsilon + xi_neg,
#         xi_pos >= 0,
#         xi_neg >= 0,
#     ]
#     problem = cp.Problem(objective, constraints)
#     problem.solve()

#     return w.value, b.value


# def gwsvr(X_train, y_train, bw):

#     def svr_section(j):
#         # Compute weights
#         weights = compute_gaussian_weights(distances, bw, j)

#         # Fit SVR with current weights
#         w, b = fit_svr_with_weights(X_train, y_train, weights)

#         # Train predictions and evaluate
#         y_j = (np.matmul(X_train[j], w) + b).item()
#         y_train_j = y_train[j]

#         # error = (y_j - y_train_j) / y_train_j
#         # print(f"Location {j}: Bandwidth = {bw}, error = {error}")

#         return w, b, y_j, y_train_j

#     results = Parallel(n_jobs=-1)(
#         delayed(svr_section)(j) for j in range(len(centroids))
#     )

#     all_w, all_b, y_pred, y_train = zip(*results)

#     return list(all_w), list(all_b), y_pred, y_train

In [8]:
import cvxpy as cp
from scipy.spatial.distance import cdist
from sklearn.metrics import mean_squared_error
from joblib import Parallel, delayed

centroids = geo_info.centroid
centroids = np.array([[point.x, point.y] for point in geo_info["centroid"]])

distances = cdist(centroids, centroids, "euclidean")


# Function to compute Gaussian weights
def compute_gaussian_weights(distances, bandwidth, j):
    weights = np.exp(-0.5 * (distances[j, :] / bandwidth) ** 2)
    return weights.reshape(-1, 1)


def fit_svr_with_weights(X_train, y_train, weights, C=1024, epsilon=0.1):
    w = cp.Variable((X_train.shape[1], 1))
    b = cp.Variable()
    xi_pos = cp.Variable((X_train.shape[0], 1))
    xi_neg = cp.Variable((X_train.shape[0], 1))

    objective = cp.Minimize(
        0.5 * cp.sum_squares(w) + cp.sum(cp.multiply(weights, xi_pos + xi_neg))
    )

    constraints = [
        y_train.reshape(-1, 1) - cp.matmul(X_train, w) - b <= epsilon + xi_pos,
        cp.matmul(X_train, w) + b - y_train.reshape(-1, 1) <= epsilon + xi_neg,
        xi_pos >= 0,
        xi_neg >= 0,
    ]
    problem = cp.Problem(objective, constraints)
    problem.solve()

    return w.value, b.value


# def gwsvr(X_train, y_train, bw):

#     all_w, all_b, y_pred = [], [], []

#     for j in range(len(centroids)):

#         # Compute weights
#         weights = compute_gaussian_weights(distances, bw, j)

#         # X_section = X_train[j].reshape(1, X_train.shape[1])
#         # y_section = y_train[j]

#         # Fit SVR with current weights
#         w, b = fit_svr_with_weights(X_train, y_train, weights)

#         # Train predictions and evaluate
#         y_j = (np.matmul(X_train[j], w) + b).item()
#         y_train_j = y_train[j]

#         # error = (y_j - y_train_j) / y_train_j
#         # print(f"Location {j}: Bandwidth = {bw}, error = {error}")

#         all_w.append(w)
#         all_b.append(b)
#         y_pred.append(y_j)

#     return all_w, all_b, y_pred, y_train


def gwsvr(X_train, y_train, bw):

    def svr_section(j):
        # Compute weights
        weights = compute_gaussian_weights(distances, bw, j)

        # X_section = X_train[j].reshape(1, X_train.shape[1])
        # y_section = y_train[j]

        # Fit SVR with current weights
        w, b = fit_svr_with_weights(X_train, y_train, weights)

        # Train predictions and evaluate
        y_j = (np.matmul(X_train[j], w) + b).item()
        y_train_j = y_train[j]

        # error = (y_j - y_train_j) / y_train_j
        # print(f"Location {j}: Bandwidth = {bw}, error = {error}")

        return w, b, y_j, y_train_j

    results = Parallel(n_jobs=-1)(
        delayed(svr_section)(j) for j in range(len(centroids))
    )

    all_w, all_b, y_pred, y_train = zip(*results)

    return list(all_w), list(all_b), y_pred, y_train

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer

from pysal.model import mgwr

# Preprocessing
X_train = X.to_numpy()
y_train = y

imputer = KNNImputer()
X_train = imputer.fit_transform(X_train)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Bandwidth Selection - One for all
y_resh = y_train.reshape((-1, 1))
gwr_chosen_selector = mgwr.sel_bw.Sel_BW(
    coords, y_resh, X_train, fixed=False, spherical=True, multi=False
)
gwr_bw = gwr_chosen_selector.search()
print("Bandwith with GWR - AIC: " + str(int(gwr_bw)))

# Specific bw selection
# mgwr_chosen_selector = mgwr.sel_bw.Sel_BW(
#     coords, y_resh, X_train, fixed=False, spherical=True, multi=True
# )
# mgwr_bw = mgwr_chosen_selector.search()
# print("Bandwith with MGWR - AIC: " + str(int(mgwr_bw)))

Bandwith with GWR - AIC: 779


In [11]:
bw = int(gwr_bw)
all_w, all_b, y_pred, y_truth = gwsvr(X_train, y_train, bw)

mse = mean_squared_error(y_truth, y_pred)
print("Mean Squared Error on train set:", mse)

Mean Squared Error on train set: 158.50093636914065


## Model Evaluation

In [30]:
from sklearn.metrics import r2_score, mean_squared_error, explained_variance_score

mse = mean_squared_error(y_truth, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_truth, y_pred)
n, p = len(y), X_train.shape[1]
adjusted_r2 = 1 - (1 - r2) * (n - 1) / (n - p - 1)
evs = explained_variance_score(y_truth, y_pred)
aic = len(y) * np.log(mse) + 2 * p

gwsvr_metrics = {
    "MSE": [mse],
    "RMSE": [rmse],
    "R^2": [r2],
    "Adj R^2": [adjusted_r2],
    "EVs": [evs],
    "AIC": [aic],
}
df_gwsvr_metrics = pd.DataFrame(gwsvr_metrics, index=["GWSVR"])

df_metrics = pd.read_csv("metrics.csv", index_col=0)
df_metrics = pd.concat([df_metrics, df_gwsvr_metrics])
df_metrics.to_csv("metrics.csv", index=True)
df_metrics

,MSE,RMSE,R^2,Adj R^2,EVs,AIC
Linear Regression,119.908203,10.950260,0.818644,0.803146,0.818644,5280.223871
GWR,128.109551,11.318549,0.806239,0.789682,0.808532,1529.338979
MESF-Queen,115.646884,10.753924,0.825089,0.808191,0.825089,5261.578263
MESF-KNN,118.783898,10.898803,0.820344,0.802988,0.820344,5290.162654
MESF-Gabriel,113.669868,10.661607,0.828079,0.811470,0.828079,5243.162637
SVR,0.914437,0.956262,0.998617,0.998499,0.998676,72.470973
GWSVR,158.500936,12.589715,0.760273,0.739788,0.762626,5578.232215


#### Spatial Autocorrelation handling

##### Moran's I on residuals 

In [28]:
from libpysal.weights import Queen
from esda.moran import Moran

residuals = y.flatten() - list(y_pred)

def calculate_morans_i(residuals, geo_info):
    w = Queen.from_dataframe(geo_info)
    w.transform = "R"
    moran = Moran(residuals, w)
    return moran.I, moran.p_sim


moran = calculate_morans_i(residuals, geo_info)
print(f"Moran's I: {moran[0]}, p-value: {moran[1]}")

Moran's I: 0.2846834897668939, p-value: 0.001


In [31]:
autocorr_metrics = {
    "Moran's I": [moran[0]],
    # "Mean MSE": [mean_mse]
}
df_autocorr_gwsvr_metrics = pd.DataFrame(autocorr_metrics, index=["GWSVR"])

df_autocorr_metrics = pd.read_csv("metrics_autocorr.csv", index_col=0)
df_autocorr_metrics = pd.concat([df_autocorr_metrics, df_autocorr_gwsvr_metrics])
df_autocorr_metrics.to_csv("metrics_autocorr.csv", index=True)
df_autocorr_metrics

,Moran's I,Mean MSE
Linear Regression,0.133607,181.599291
GWR,0.091858,192.927322
MESF-Queen,0.111243,283.972372
MESF-KNN,0.099513,233.907563
MESF-Gabriel,0.040453,234.958120
SVR,0.098604,280.698096
GWSVR,0.284683,NaN
